In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from cleaner_pretreinamento import Cleaner, Corretor
from tqdm.auto import tqdm
import pickle

### Objetos

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_tipos_objetos_raw_completo.csv')
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, 'text', cased=True, accents=True)
dados

In [ ]:
dados.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsGold/df_tipos_objetos_cased_numbers.csv", index=False)

### Indicios

In [2]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_gold_hab_atualizada.csv')
dados

FileNotFoundError: [Errno 2] No such file or directory: '/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsGold/dataset_gold_hab_atualizada.csv'

In [ ]:
['text', 'certidao_protesto', 'certificado_boas_praticas',
       'comprovante_localizacao', 'idoneidade_financeira', 'integralizado',
       'licenca_ambiental', 'n_min_max_limitacao_atestados'],

In [ ]:
print(dados['certidao_protesto'].value_counts())
print(dados['certificado_boas_praticas'].value_counts())
print(dados['comprovante_localizacao'].value_counts())
print(dados['idoneidade_financeira'].value_counts())
print(dados['integralizado'].value_counts())
print(dados['licenca_ambiental'].value_counts())
print(dados['n_min_max_limitacao_atestados'].value_counts())

In [ ]:
columns = ['certidao_protesto', 'certificado_boas_praticas', 'comprovante_localizacao', 'idoneidade_financeira', 'integralizado', 'licenca_ambiental', 'n_min_max_limitacao_atestados']
print(columns)

In [ ]:
for coluna in columns:
    dados[coluna] = 0

In [ ]:
dados

In [ ]:
for indice in dados.index:
    for label in eval(dados['annotation'][indice]):
        if label in columns:
            dados.at[indice, label] = 1

In [ ]:
dados

In [ ]:
dados.drop(columns=['annotation'], inplace=True)
dados

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, 'text', True, True)

In [ ]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
X_train, y_train, X_test, y_test = iterative_train_test_split(dados.values, dados[dados.columns[1:]].values, test_size = 0.25)
X_test, y_test, X_val, y_val = iterative_train_test_split(X_test, y_test, test_size = 0.5)

In [ ]:
treino = pd.DataFrame(X_train, columns=dados.columns)
teste = pd.DataFrame(X_test, columns=dados.columns)
validacao = pd.DataFrame(X_val, columns=dados.columns)

In [ ]:
treino

In [ ]:
teste

In [ ]:
validacao

In [ ]:
treino.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/dataset_bid_notices_gold_treino.csv', index=False)
teste.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/dataset_bid_notices_gold_teste.csv', index=False)
validacao.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/dataset_bid_notices_gold_validacao.csv', index=False)

### Ner Indicios

In [ ]:
treino = pickle.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/bid_notices_weak/dataset_bid_notices_weak_hab_clean_treino_chunk_more.pkl', 'rb'))
teste = pickle.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/bid_notices_weak/dataset_bid_notices_weak_hab_clean_teste_chunk_more.pkl', 'rb'))
validacao = pickle.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERT/Datasets/Indicios/bid_notices_weak/dataset_bid_notices_weak_hab_clean_validacao_chunk_more.pkl', 'rb'))

### Dataset Perplexidade

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/PreTreinamento/dfPreTreinamento-base-uncased.csv')
dados

In [ ]:
teste = dados['text'].tolist()[:100000]
treinamento = dados['text'].tolist()[100000:]

In [ ]:
with open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/PreTreinamento/teste.txt', 'w') as file:
    for texto in teste:
        file.write(texto + '\n')

In [ ]:
with open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/PreTreinamento/HelBERT-base-uncased/treino.txt', 'w') as file:
    for texto in treinamento:
        file.write(texto + '\n')

### MLM Sentences

In [ ]:
!pip install unidecode

In [ ]:
import re, json
from unidecode import unidecode

In [ ]:
palavrasIRb = json.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/tesauroIRB3Palavras.json'))
palavrasIRb

In [ ]:
lista_palavras = []
for palavra in palavrasIRb:
    lista_palavras.append(palavra['Palavra'])
    lista_palavras.append(palavra['Relacionada'])
    

In [ ]:
lista_palavras = list(set(lista_palavras))
lista_palavras